### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [5]:
data = pd.read_csv('/content/gdrive/My Drive/AIML/Labs/R6-Internal/prices.csv')
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
data1000 = data.head(1000)
data1000.shape

(1000, 5)

### Divide the data into train and test sets

In [10]:
from sklearn.model_selection import train_test_split

X = data1000.copy(deep=True)
Y = data1000['close']
X.drop(columns='close', inplace=True)

print(X.shape)
print(Y.shape)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=7)

(1000, 4)
(1000,)


In [11]:
X.head()

,open,low,high,volume
0,123.430000,122.309998,126.250000,2163600.0
1,125.239998,119.940002,125.540001,2386400.0
2,116.379997,114.930000,119.739998,2489500.0
3,115.480003,113.500000,117.440002,2006300.0
4,117.010002,114.089996,117.330002,1408600.0


In [12]:
Y.head()

0    125.839996
1    119.980003
2    114.949997
3    116.620003
4    114.970001
Name: close, dtype: float64

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
x = tf.placeholder(dtype=tf.float32, shape=[None,4], name='x-input')
x_n = tf.layers.batch_normalization(x,training=True)
y_ = tf.placeholder(dtype=tf.float32, shape=[None], name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), dtype=tf.float32, name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x_n, W), b, name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_), name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

training_epochs = 100

In [19]:
for epoch in range(training_epochs):
  
  _, train_loss = sess.run([train_op,loss], feed_dict={x:X_train,
                                                       y_:Y_train})
  
  if epoch % 10 == 0:
     print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 8370.378)
('Training loss at step: ', 10, ' is ', 4992.165)
('Training loss at step: ', 20, ' is ', 3689.0046)
('Training loss at step: ', 30, ' is ', 4038.9663)
('Training loss at step: ', 40, ' is ', 4018.743)
('Training loss at step: ', 50, ' is ', 3747.9004)
('Training loss at step: ', 60, ' is ', 3711.2063)
('Training loss at step: ', 70, ' is ', 3708.5874)
('Training loss at step: ', 80, ' is ', 3701.5264)
('Training loss at step: ', 90, ' is ', 3696.081)


In [20]:
  train_loss

3702.1038

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [21]:
sess.run(W)

array([[ 2.8714082 ],
       [ 0.8775928 ],
       [-3.4261193 ],
       [ 0.02576405]], dtype=float32)

In [22]:
sess.run(b)

array([51.513824], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [0]:
Y_pred = sess.run(y, feed_dict={x:X_test})

In [24]:
Y_test.dtype

dtype('float64')

In [25]:
from sklearn.metrics import mean_squared_error
test_loss = mean_squared_error(Y_test,Y_pred)
test_loss

2828.6741618732212

In [0]:
##tf.reduce_mean(tf.square(Y_test-Y_pred))

In [28]:
difference = train_loss - test_loss
difference

873.4295978924038

In [29]:
'''for epoch in range(training_epochs):
  
  _, test_loss_1 = sess.run([train_op,loss], feed_dict={x:X_test,
                                                       y_:Y_test})
  
  if epoch % 10 == 0:
     print ('Test loss at step: ', epoch, ' is ', test_loss_1)'''

"for epoch in range(training_epochs):\n  \n  _, test_loss_1 = sess.run([train_op,loss], feed_dict={x:X_test,\n                                                       y_:Y_test})\n  \n  if epoch % 10 == 0:\n     print ('Test loss at step: ', epoch, ' is ', test_loss_1)"

<h4> The difference between training and testing loss is 896.71 <h4>

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [31]:
model.fit(X_train,Y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 215us/step - loss: nan
Epoch 2/100
700/700 [==============================] - 0s 40us/step - loss: nan
Epoch 3/100
700/700 [==============================] - 0s 36us/step - loss: nan
Epoch 4/100
700/700 [==============================] - 0s 34us/step - loss: nan
Epoch 5/100
700/700 [==============================] - 0s 34us/step - loss: nan
Epoch 6/100
700/700 [==============================] - 0s 34us/step - loss: nan
Epoch 7/100
700/700 [==============================] - 0s 34us/step - loss: nan
Epoch 8/100
700/700 [==============================] - 0s 36us/step - loss: nan
Epoch 9/100
700/700 [==============================] - 0s 33us/step - loss: nan
Epoch 10/100
700/700 [==============================] - 0s 33us/step - loss: nan
Epoch 11/100
700/700 [==============================] - 0s 30us/step - loss: nan
Epoch 12/100
700/700 [==============================] - 0s 36us/step - loss: nan
Epoch 13/100
700/700 [==============

In [32]:
type(X_train)

pandas.core.frame.DataFrame

In [33]:
type(Y_train)

pandas.core.series.Series

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [34]:
iris_df = pd.read_csv('/content/gdrive/My Drive/AIML/Labs/R6-Internal/Iris.csv')
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [35]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Splitting the data into feature set and target set

In [0]:
X_iris = iris_df.copy(deep=True)
Y_iris = X_iris['Species']
X_iris.drop(columns=['Species','Id'],inplace=True)

In [37]:
X_iris.shape

(150, 4)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [38]:
'''from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_iris = pd.DataFrame(le.fit_transform(Y_iris))'''

Y_iris = pd.get_dummies(Y_iris)
Y_iris

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [39]:
Y_iris.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [0]:
X_iris_train,X_iris_test,Y_iris_train,Y_iris_test = train_test_split(X_iris,Y_iris,test_size=0.3,random_state=7)

In [41]:
X_iris_train.shape

(105, 4)

In [42]:
Y_iris_test

,Iris-setosa,Iris-versicolor,Iris-virginica
149,0,0,1
84,0,1,0
40,1,0,0
66,0,1,0
106,0,0,1
41,1,0,0
52,0,1,0
94,0,1,0
11,1,0,0
51,0,1,0


### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
import tensorflow as tf1

tf1.reset_default_graph()
tf1.set_random_seed(42)

#Initialize Sequential model
model_iris = tf1.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
##model_iris.add(tf.keras.layers.Reshape((4,),))


model_iris.add(tf.keras.layers.Dense(3))
#Normalize the data
model_iris.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))



#Add Dense Layer which provides 10 Outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(10, activation='sigmoid'))

model_iris.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#Add OUTPUT layer
model_iris.add(tf.keras.layers.Dense(3, activation='softmax'))


#Comile the model
model_iris.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [63]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Fitting the model and predicting 

In [64]:
model_iris.fit(X_iris_train,Y_iris_train,          
          validation_data=(X_iris_test,Y_iris_test),
          epochs=30)

ValueError: ignored

### Report Accuracy of the predicted values